# Kinematics of the Bit
**A companion notebook to "Graduate Quantum Information Science"**

In [ ]:
import doctest

import  xzcbnumpy as np
from scipy.linalg import expm

# magic word for producing visualizations in notebook
%matplotlib inline


## 
# Originally for 
# Physics 116: Quantum Information Science
# Dartmouth College, Spring 2022
# Instructor: James Whitfield (james.d.whitfield@dartmouth.edu) 
# Teaching Assistant: AJ Cressman (anthony.j.cressman.gr@dartmouth.edu)

# JDWhitfield 2024

: 

This notebook is meant to illustrate the probability-first approach to quantum used in "Graduate Quantum Information Science". Specifically, we are concerned with the kinematics of probability density vectors for the two outcome situation. The notation and examples largely follow the associated appendix on the kinematics of the bit.

We consider states first and continue on to their transformations. 

### Table of contents
* [Probability density vectors for bits](#states)
* [Kinematially allowed transformations](#transforms)

# Probability density vectors for bits <a class="anchor" id="states"></a>

In [2]:
def is_probability_density_vector(v):
    """Checks if a given array represents a probability density vector.

    Probability density vectors must be one-dimensional (vectors), with real, non-negative entries that sum to 1.

    Args:
        v ([float]): A one-dimensional array of numbers.

    Returns:
        (bool): True if the array represents a probability density vector, False otherwise

    Examples of probability density vectors:

    >>> is_probability_density_vector([1, 0])
    True
    >>> is_probability_density_vector([0.25, 0.25, 0.25, 0.25])
    True
    >>> is_probability_density_vector(np.array([1, 0]))
    True
    >>> is_probability_density_vector(np.matrix([1, 0]))
    True
    >>> is_probability_density_vector([[1], [0]])
    True
    >>> is_probability_density_vector(np.matrix([[0], [1]]))
    True
    >>> is_probability_density_vector([0.5, 0.5])
    True
    >>> is_probability_density_vector([[1, 0]])
    True
    >>> is_probability_density_vector([1 + 0j, 0])
    True

    Examples of arrays that are not probability density vectors:

    Not one-dimensional array
    >>> is_probability_density_vector([[1, 0], [0, 0]])
    False

    Not all real
    >>> is_probability_density_vector([1 + 1j, 0])
    False


    Not all positive
    >>> is_probability_density_vector([1.1, -0.1])
    False
    >>> is_probability_density_vector([-1 + 0j, 0])
    False

    Not normal
    >>> is_probability_density_vector([1, 1])
    False
    """
    # treat v as a numpy array, collapse "empty" dimensions
    v = np.squeeze(np.array(v))
    # check if one-dimensional
    if v.ndim != 1:
        return False
    # check if all real
    if not np.isreal(v).all():
        return False
    # check if all positive
    v = np.real(v)
    if (v < 0).any():
        return False
    # check normalization
    if not np.isclose(np.sum(v), 1):
        return False
    return True


# uncomment to run tests:
# doctest.testmod()

In [3]:
# Bernoulli parameters (adjust as needed)
p = 0.75  # probability [0, 1]
assert np.isreal(p) and p >= 0 and p <= 1

In [4]:
# Turn our probability p (set at the top) into a probability density vector
vecp = np.matrix([[p], [1 - p]])
print(f"vecp: \n{vecp}")
# vecp and its transpose should both be probability density vectors
assert is_probability_density_vector(vecp)
assert is_probability_density_vector(vecp.T)

# bias parameter
b = p - 1 / 2

vecp: 
[[0.8]
 [0.2]]


In [5]:
# Useful matrices to represent change of basis of 2D probability vectors
I = np.eye(2)  # the identity
P12 = np.matrix([[0, 1], [1, 0]])  # permutation (1 2) -- change of basis for a bit

# State transformations <a class="anchor" id="transforms"></a>

## Change of basis for a bit

In [6]:
# bistochastic change of basis for a bit

# NOTE: the ": float" is a type hint.
# Using them is *not* necessary, but they can help document what your function does
# (and check that what you've done is correct!).
def change_of_basis_matrix_2d(w: float):
    """Given a weight w, returns a change of basis matrix for 2D probability vectors

    Args:
        w (float): The weight given to permutation P12 (in the range [0, 1])

    Returns:
        M (np.matrix): A 2x2 matrix representing a change of basis

    Raises:
        ValueError: if the weight w is not real and in the range [0, 1]

    Examples:
    >>> change_of_basis_matrix_2d(0)
    matrix([[1., 0.],
            [0., 1.]])

    >>> change_of_basis_matrix_2d(1)
    matrix([[0., 1.],
            [1., 0.]])

    >>> change_of_basis_matrix_2d(0.5)
    matrix([[0.5, 0.5],
            [0.5, 0.5]])

    >>> change_of_basis_matrix_2d(0+0j)
    matrix([[1., 0.],
            [0., 1.]])

    >>> change_of_basis_matrix_2d(-1)
    Traceback (most recent call last):
    ...
    ValueError: w must be between 0 and 1 (inclusive), given: -1

    >>> change_of_basis_matrix_2d(0+1j)
    Traceback (most recent call last):
    ...
    ValueError: w must be real, given: 1j

    """
    if not np.isreal(w):
        raise ValueError(f"w must be real, given: {w}")
    w = np.real(w)  # remove "+0j" imaginary component
    if w < 0 or w > 1:
        raise ValueError(f"w must be between 0 and 1 (inclusive), given: {w}")
    M = w * P12 + (1 - w) * I
    return M


# uncomment to run tests:
# doctest.testmod()

In [7]:
assert is_probability_density_vector(vecp)
# NOTE: the "@" symbol is how numpy expresses matrix multiplication
assert is_probability_density_vector(change_of_basis_matrix_2d(0) @ vecp)
assert is_probability_density_vector(change_of_basis_matrix_2d(1) @ vecp)
assert is_probability_density_vector(change_of_basis_matrix_2d(0.5) @ vecp)

## Markov matrices for probability state transformations

In [8]:
# general stochastic transformations of a bit
def general_markov_transform_2d(w1: float, w2: float):
    """Given two weights w1 and w2, return the general (2D) Markov transform

    Args:
        w1 (float): The weight of no permutation (in the range [0, 1])
        w2 (float): The weight of no permutation (in the range [0, 1])

    Returns:
        M (np.matrix): A 2x2 matrix representing a general Markov transform

    Raises:
        ValueError: if the weights (w1 and w2) are not both real and in the range [0, 1]

    Examples:

    >>> general_markov_transform_2d(0, 0)
    matrix([[0., 0.],
            [1., 1.]])

    >>> general_markov_transform_2d(0, 1)
    matrix([[0., 1.],
            [1., 0.]])

    >>> general_markov_transform_2d(0.5, 0.5)
    matrix([[0.5, 0.5],
            [0.5, 0.5]])

    Failing examples:

    >>> general_markov_transform_2d(0+1j, 0)
    Traceback (most recent call last):
    ...
    ValueError: w1 must be real, given: 1j

    >>> general_markov_transform_2d(0, 0+1j)
    Traceback (most recent call last):
    ...
    ValueError: w2 must be real, given: 1j

    >>> general_markov_transform_2d(2, 0)
    Traceback (most recent call last):
    ...
    ValueError: w1 must be between 0 and 1 (inclusive), given: 2

    >>> general_markov_transform_2d(0, 2)
    Traceback (most recent call last):
    ...
    ValueError: w2 must be between 0 and 1 (inclusive), given: 2
    """
    # w1 checks
    if not np.isreal(w1):
        raise ValueError(f"w1 must be real, given: {w1}")
    w1 = np.real(w1)  # remove "+0j" imaginary component
    if w1 < 0 or w1 > 1:
        raise ValueError(f"w1 must be between 0 and 1 (inclusive), given: {w1}")
    # w2 checks
    if not np.isreal(w2):
        raise ValueError(f"w2 must be real, given: {w2}")
    w2 = np.real(w2)  # remove "+0j" imaginary component
    if w2 < 0 or w2 > 1:
        raise ValueError(f"w2 must be between 0 and 1 (inclusive), given: {w2}")

    M = np.matrix([[w1, w2], [1.0 - w1, 1.0 - w2]])
    return M


# uncomment to run tests:
# doctest.testmod()

In [9]:
assert is_probability_density_vector(general_markov_transform_2d(0, 0) @ vecp)
assert is_probability_density_vector(general_markov_transform_2d(1, 0) @ vecp)
assert is_probability_density_vector(general_markov_transform_2d(0, 1) @ vecp)
assert is_probability_density_vector(general_markov_transform_2d(0.5, 0.5) @ vecp)

## Continuous time transformations

In [11]:
# M is a probability transition matrix
# while W is a rate matrix i.e. W = M/dt

W = np.abs(np.random.randn(2, 2))  # Wij nonzero, no other assumptions
D = np.diag(np.sum(W, axis=0))  # sum along the row, then turn into diagonal matrix

# Here axis=0 or axis=1 will change the stochastic matrix from left to right
# Feel free to test this out

L = W - D

print(f"L generates changes in time: \n{L}")

# Students: Play with this value! Does it have to be positive?
t = 1.0

print(f"\nvecp is a probability density vector")
print(is_probability_density_vector(vecp))

print(f"\nL generates changes in time on the right")
print(is_probability_density_vector(expm(L * t) @ vecp))

print(f"\nBut not on the left")
print(is_probability_density_vector(vecp.T * expm(L * t)))

print(f"\nIts transpose doesn't work on the right")
print(is_probability_density_vector(expm(L.T * t) @ vecp))

print(f"\nBut it does work on the left")
print(is_probability_density_vector((vecp.T) @ expm(L.T * t)))

L generates changes in time: 
[[-0.69415752  0.20008196]
 [ 0.69415752 -0.20008196]]

vecp is a probability density vector
True

L generates changes in time on the right
True

But not on the left
False

Its transpose doesn't work on the right
False

But it does work on the left
True


In [2]:
# AJ Cressman 2022
# JDWhitfield 2022
# JDWhitfield 2024